In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

$env:MLFLOW_TRACKING_URI="https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow"
$env:MLFLOW_TRACKING_USERNAME="rppawar491"
$env:MLFLOW_TRACKING_PASSWORD="024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12e"

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "rppawar491"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "490e6ae7bb9405fa77d80a472fcda7758b0d1abd"

### Entity

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    x_test_data_path : Path
    y_test_data_path : Path
    metric_file_name : Path
    model_path : Path
    all_params : dict
    target_column :str
    mlflow_uri : str

### Configuration manager

In [21]:
from Bone_marrow_survival_prediction.constants import *
from Bone_marrow_survival_prediction.utils.comman import read_yaml, create_directories
from Bone_marrow_survival_prediction import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SupportVectorclassifier
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            metric_file_name = config.metric_file_name,
            model_path = config.model_path,
            all_params = params,
            target_column = schema,
            mlflow_uri = "https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow"
            
        )
        
        return model_evaluation_config

### Component

In [22]:
def setup_dagshub_mlflow_tracking(token):
    
    try:
       
        dagshub_uri = "https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow"
        
        os.environ['MLFLOW_TRACKING_URI'] = dagshub_uri
        os.environ['MLFLOW_TRACKING_TOKEN'] = token
        
        mlflow.set_tracking_uri(dagshub_uri)
        
        logger.info(f"DagsHub MLflow Tracking Configured Successfully")
    except Exception as e:
        logger.exception(e)
        raise e

In [23]:
import mlflow
import joblib
import pandas as pd
import numpy as np
from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, f1_score


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):

        try:
            accuracy = accuracy_score(actual, pred)
            precision = precision_score(actual, pred, average='weighted')
            f1 = f1_score(actual, pred, average='weighted')
            return accuracy, precision, f1
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            return 0, 0, 0
    
    def log_into_mlflow(self, access_token):
        
        try:
            
            setup_dagshub_mlflow_tracking(access_token)
            test_df = pd.read_csv(self.config.x_test_data_path)
            test_labels_df = pd.read_csv(self.config.y_test_data_path)
            
            
            X_test = test_df
            y_test = test_labels_df
            
           
            model = joblib.load(self.config.model_path)
            
           
            y_pred = model.predict(X_test)
            
           
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            
            with mlflow.start_run():
               
                mlflow.log_metrics({
                    "accuracy": accuracy,
                    "precision": precision,
                    "f1_score": f1
                })
                
                
                mlflow.sklearn.log_model(
                    model, 
                    "model", 
                    registered_model_name="BoneMarrowSurvivalModel"
                )
                
                logger.info("Model and metrics logged successfully")
                
                return {
                    "accuracy": accuracy,
                    "precision": precision,
                    "f1_score": f1
                }
        
        except Exception as e:
            logger.exception(e)
            raise e

### Pipeline

In [24]:
try:
        DAGSHUB_ACCESS_TOKEN = "490e6ae7bb9405fa77d80a472fcda7758b0d1abd"
        config = ConfigurationManager()
        model_evaluation_config = config.get_model_evaluation_config()
        model_evaluation = ModelEvaluation(config=model_evaluation_config)
        metrics = model_evaluation.log_into_mlflow(DAGSHUB_ACCESS_TOKEN)
except Exception as e:
    raise e

[ 2025-01-25 20:54:50,832: INFO: comman: yaml file: config\config.yaml loaded successfully]
[ 2025-01-25 20:54:50,840: INFO: comman: yaml file: params.yaml loaded successfully]
[ 2025-01-25 20:54:50,848: INFO: comman: yaml file: schema.yaml loaded successfully]
[ 2025-01-25 20:54:50,849: INFO: comman: created directory at: artifacts]
[ 2025-01-25 20:54:50,849: INFO: comman: created directory at: artifacts/model_evaluation]
[ 2025-01-25 20:54:50,854: INFO: 4195662002: DagsHub MLflow Tracking Configured Successfully]


2025/01/25 20:54:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'BoneMarrowSurvivalModel' already exists. Creating a new version of this model...
2025/01/25 20:55:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BoneMarrowSurvivalModel, version 6


[ 2025-01-25 20:55:03,582: INFO: 2429907925: Model and metrics logged successfully]


Created version '6' of model 'BoneMarrowSurvivalModel'.


🏃 View run luxuriant-eel-944 at: https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow/#/experiments/0/runs/3385c9fd24e241c2b3532bca74c1b492
🧪 View experiment at: https://dagshub.com/rppawar491/Bone_marrow_surival_prediction.mlflow/#/experiments/0
